<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Steam games dataset 2024
</div>

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
<strong>Extraccion de informacion</strong>

<p>
Recuperdor de información, para saber  que tipo de contenido se desea como resumenes, detallesm reseñas, edades
</p>
</div>

In [2]:
import pandas as pd
import re

In [3]:
pd.set_option('display.max_colwidth', 2000)
pd.options.display.max_rows=100

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Realizamos esta linea ya que teniamos que definir cada columna con su tipo de dato para no tener problemas luego
</div>

In [4]:
dtype_dict = {
    'AppID': 'int64',
    'name': 'object',
    'release_date': 'object',
    'required_age': 'object',
    'price': 'float64',
    'dlc_count': 'float64',
    'detailed_description': 'object',
    'about_the_game': 'object',
    'short_description': 'object',
    'reviews': 'object',
    'header_image': 'object',
    'website': 'object',
    'support_url': 'object',
    'support_email': 'object',
    'windows': 'object',
    'mac': 'object',
    'linux': 'object',
    'metacritic_score': 'object',
    'metacritic_url': 'object',
    'achievements': 'object',
    'recommendations': 'object',
    'notes': 'object',
    'supported_languages': 'object',
    'full_audio_languages': 'object',
    'packages': 'object',
    
    'Unnamed: 87': 'float64',
    'Unnamed: 88': 'float64',
    'Unnamed: 89': 'float64'
}


<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Leemos el dataset 
</div>

In [5]:
df = pd.read_csv('games_may2024_cleaned.csv',  dtype=dtype_dict, low_memory=False)

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Elegimos las columnas que queremos extraer su informacion
</div>

In [6]:
df = df[["detailed_description", "about_the_game", "short_description", "reviews", "supported_languages", "full_audio_languages"]]

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Realizamos estas 2 celdas ya que no queremos los NAN en cada fila de cada columna
</div>

In [7]:
df_cleaned = df.dropna()

In [8]:
df_cleaned.to_csv('games_may2024_cleaned_text.csv', index=False)

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Podemos ver el df sin los NAN en cada fila de cada columna 
</div>

In [9]:
df_cleaned

,detailed_description,about_the_game,short_description,reviews,supported_languages,full_audio_languages
2,"The most-played game on Steam. Every day, millions of players worldwide enter battle as one of over a hundred Dota heroes. And no matter if it's their 10th hour of play or 1,000th, there's always something new to discover. With regular updates that ensure a constant evolution of gameplay, features, and heroes, Dota 2 has truly taken on a life of its own. One Battlefield. Infinite Possibilities. When it comes to diversity of heroes, abilities, and powerful items, Dota boasts an endless array—no two games are the same. Any hero can fill multiple roles, and there's an abundance of items to help meet the needs of each game. Dota doesn't provide limitations on how to play, it empowers you to express your own style. All heroes are free. Competitive balance is Dota's crown jewel, and to ensure everyone is playing on an even field, the core content of the game—like the vast pool of heroes—is available to all players. Fans can collect cosmetics for heroes and fun add-ons for the world they inhabit, but everything you need to play is already included before you join your first match. Bring your friends and party up. Dota is deep, and constantly evolving, but it's never too late to join. Learn the ropes playing co-op vs. bots. Sharpen your skills in the hero demo mode. Jump into the behavior- and skill-based matchmaking system that ensures you'll be matched with the right players each game.","The most-played game on Steam. Every day, millions of players worldwide enter battle as one of over a hundred Dota heroes. And no matter if it's their 10th hour of play or 1,000th, there's always something new to discover. With regular updates that ensure a constant evolution of gameplay, features, and heroes, Dota 2 has truly taken on a life of its own. One Battlefield. Infinite Possibilities. When it comes to diversity of heroes, abilities, and powerful items, Dota boasts an endless array—no two games are the same. Any hero can fill multiple roles, and there's an abundance of items to help meet the needs of each game. Dota doesn't provide limitations on how to play, it empowers you to express your own style. All heroes are free. Competitive balance is Dota's crown jewel, and to ensure everyone is playing on an even field, the core content of the game—like the vast pool of heroes—is available to all players. Fans can collect cosmetics for heroes and fun add-ons for the world they inhabit, but everything you need to play is already included before you join your first match. Bring your friends and party up. Dota is deep, and constantly evolving, but it's never too late to join. Learn the ropes playing co-op vs. bots. Sharpen your skills in the hero demo mode. Jump into the behavior- and skill-based matchmaking system that ensures you'll be matched with the right players each game.","Every day, millions of players worldwide enter battle as one of over a hundred Dota heroes. And no matter if it's their 10th hour of play or 1,000th, there's always something new to discover. With regular updates that ensure a constant evolution of gameplay, features, and heroes, Dota 2 has taken on a life of its own.","“A modern multiplayer masterpiece.” 9.5/10 – Destructoid “Once you start to learn its secrets, there’s a wild and exciting variety of play here that’s unmatched, even by its peers.” 9.4/10 – IGN “Dota 2 is possibly the only competitive free-to-play game that is totally uncompromised by its business model.” 90/100 – PC Gamer","['Bulgarian', 'Czech', 'Danish', 'Dutch', 'English', 'Finnish', 'French', 'German', 'Greek', 'Hungarian', 'Italian', 'Japanese', 'Korean', 'Norwegian', 'Polish', 'Portuguese - Portugal', 'Portuguese - Brazil', 'Romanian', 'Russian', 'Simplified Chinese', 'Spanish - Spain', 'Swedish', 'Thai', 'Traditional Chinese', 'Turkish', 'Ukrainian', 'Spanish - Latin America', 'Vietnamese']","['English', 'Korean', 'Simplified Chinese', 'Vietn

<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Realizamos la limpieza profunda
</div>

In [10]:
%pip install emoji

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
def clean_df(df):
    if df is None:
        return None
    df_copy = df.copy()
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U00002600-\U000026FF"  # Miscellaneous Symbols
                           u"\U00002700-\U000027BF"  # Dingbats
                           "]+", flags=re.UNICODE)
    # Aplicar str.replace() a cada columna de texto en el DataFrame de forma segura
    for columna in df_copy.columns:
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace('•', '', regex=False)
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace('◆', '', regex=False)
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace('™', '', regex=False) # no se esta limpiando este al parecer
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace('®', '', regex=False) # no se esta limpiando este al parecer
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace('©', '', regex=False) # no se esta limpiando este al parecer
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace('█', '', regex=False)
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace('▒', '', regex=False)
        df_copy.loc[:, columna] = df_cleaned[columna].astype(str).str.replace('__', '', regex=True)
        df_copy.loc[:, columna] = df_cleaned[columna].astype(str).str.replace('……', '', regex=True)
        df_copy.loc[:, columna] = df_cleaned[columna].astype(str).str.replace('†', '', regex=True)
        df_copy.loc[:, columna] = df_cleaned[columna].astype(str).str.replace('※', '', regex=True)
        df_copy.loc[:, columna] = df_cleaned[columna].astype(str).str.replace('', '', regex=True)
        df_copy.loc[:, columna] = df_cleaned[columna].astype(str).str.replace('£', '', regex=True)
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace('⊰⊱', '', regex=False)
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace('๑̀́و', '', regex=False)
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace("'", '', regex=False)
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace("...", '', regex=False)
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace("-", '', regex=False)
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace("—", '', regex=False)
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace("_", '', regex=False)
        df_copy.loc[:, columna] = df_copy[columna].astype(str).apply(lambda x: emoji_pattern.sub(r'', x))
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace('___________', '', regex=False)
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.replace(r'[,.:;=()*+&^%$#@!{}~\[\]<>?/|\\\-\'"«»–“”]', '', regex=True)
        df_copy.loc[:, columna] = df_copy[columna].astype(str).str.lower()
        df_copy[columna] = df_copy[columna].apply(lambda x: x.replace('—', '') if isinstance(x, str) else x)
        df_copy[columna] = df_copy[columna].apply(lambda x: x.replace('_', '') if isinstance(x, str) else x)

    return df_copy

In [12]:
df_cleaned = clean_df(df_cleaned)

In [13]:
df_cleaned

,detailed_description,about_the_game,short_description,reviews,supported_languages,full_audio_languages
2,the mostplayed game on steam every day millions of players worldwide enter battle as one of over a hundred dota heroes and no matter if its their 10th hour of play or 1000th theres always something new to discover with regular updates that ensure a constant evolution of gameplay features and heroes dota 2 has truly taken on a life of its own one battlefield infinite possibilities when it comes to diversity of heroes abilities and powerful items dota boasts an endless arrayno two games are the same any hero can fill multiple roles and theres an abundance of items to help meet the needs of each game dota doesnt provide limitations on how to play it empowers you to express your own style all heroes are free competitive balance is dotas crown jewel and to ensure everyone is playing on an even field the core content of the gamelike the vast pool of heroesis available to all players fans can collect cosmetics for heroes and fun addons for the world they inhabit but everything you need to play is already included before you join your first match bring your friends and party up dota is deep and constantly evolving but its never too late to join learn the ropes playing coop vs bots sharpen your skills in the hero demo mode jump into the behavior and skillbased matchmaking system that ensures youll be matched with the right players each game,the mostplayed game on steam every day millions of players worldwide enter battle as one of over a hundred dota heroes and no matter if its their 10th hour of play or 1000th theres always something new to discover with regular updates that ensure a constant evolution of gameplay features and heroes dota 2 has truly taken on a life of its own one battlefield infinite possibilities when it comes to diversity of heroes abilities and powerful items dota boasts an endless arrayno two games are the same any hero can fill multiple roles and theres an abundance of items to help meet the needs of each game dota doesnt provide limitations on how to play it empowers you to express your own style all heroes are free competitive balance is dotas crown jewel and to ensure everyone is playing on an even field the core content of the gamelike the vast pool of heroesis available to all players fans can collect cosmetics for heroes and fun addons for the world they inhabit but everything you need to play is already included before you join your first match bring your friends and party up dota is deep and constantly evolving but its never too late to join learn the ropes playing coop vs bots sharpen your skills in the hero demo mode jump into the behavior and skillbased matchmaking system that ensures youll be matched with the right players each game,every day millions of players worldwide enter battle as one of over a hundred dota heroes and no matter if its their 10th hour of play or 1000th theres always something new to discover with regular updates that ensure a constant evolution of gameplay features and heroes dota 2 has taken on a life of its own,a modern multiplayer masterpiece 9510 destructoid once you start to learn its secrets there’s a wild and exciting variety of play here that’s unmatched even by its peers 9410 ign dota 2 is possibly the only competitive freetoplay game that is totally uncompromised by its business model 90100 pc gamer,bulgarian czech danish dutch english finnish french german greek hungarian italian japanese korean norwegian polish portuguese portugal portuguese brazil romanian russian simplified chinese spanish spain swedish thai traditional chinese turkish ukrainian spanish latin america vietnamese,english korean simplified chinese vietnamese
7,garrys mod is a physics sandbox there arent any predefined aims or goals we give you the tools and leave you to play you spawn objects and weld them together to create your own contraptions whether thats a car a rocket a catapult or somethi

In [14]:
# Asumiendo que df_cleaned es tu DataFrame limpio
df_cleaned.to_csv('/home/natashalop/Escritorio/games_may2024_cleaned.csv', index=False)